### Importing Packages

In [1]:
!unzip -o mosque_dataset.zip

Archive:  mosque_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of mosque_dataset.zip or
        mosque_dataset.zip.zip, and cannot find mosque_dataset.zip.ZIP, period.


In [2]:
from model import GeneratorGAN, DiscriminatorGAN
from train import trainDiscriminator, trainGenerator, trainProcess, trainResults
from test import testResults, testPlotting
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
import torchvision.utils as vutils
from torchvision import transforms
import torchvision.datasets as dset

### Initializing GPU- we wanna make sure to use this if available

In [3]:
print(f'CUDA available: {torch.cuda.is_available()}')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CUDA available: True


In [4]:
import os
from PIL import Image
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.images = [os.path.join(root, file) for file in os.listdir(root) if file.endswith(('jpg', 'jpeg', 'png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0


In [5]:
# resize to 64 x 64 
dataset = ImageDataset(root='mosque_dataset',
                       transform=transforms.Compose([
                           transforms.Resize(64),
                           transforms.CenterCrop(64),
                           transforms.ToTensor(),
                       ]))

train_size = int(.7 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=True, num_workers=2, drop_last=True)

FileNotFoundError: [Errno 2] No such file or directory: 'mosque_dataset'

### Displaying Real Images

In [ ]:
for i, (images, _) in enumerate(train_loader):
    images = images.to(device)[:64] # get only half of the 128 images
    break
plt.figure(figsize = (8, 8))
plt.title("Real Images from Training Data")
plt.axis('off')
plt.imshow(np.transpose(vutils.make_grid(images, padding= True).cpu(), (1, 2, 0))) # rearrange to height, width, channels

### Weight Initialization

In [ ]:
def w_init(m):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        nn.init.normal_(m.weight.data, 0, .02) # 0 mean 0.02 stdev
    if isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1, .02) # 1 mean .02 stdev
        nn.init.constant_(m.bias.data, 0) #some constant for bn

### Initializing and Debugging Generator Model

In [ ]:
G = GeneratorGAN(kernSize = 5, padVal = 2)
G = G.to(device)
G.apply(w_init)

# testing random vector to ensure correct output
fixed_noise = torch.rand(128, 100, 1, 1)
fixed_noise = fixed_noise.to(device)
with torch.no_grad():
    output = G(fixed_noise)
output.shape

### Initializing and Debugging Discriminator Model

In [ ]:
D = DiscriminatorGAN(kernSize = 5, padVal = 2)
D = D.to(device)
D.apply(w_init)
# testing random image to ensure correct output
sample_image = dataset[0][0]
sample_image = sample_image.unsqueeze(0)
sample_image = sample_image.to(device)
with torch.no_grad():
    output = D(sample_image)
output.shape

### Optimizing Number of Epochs

In [ ]:
# # num_epochs should be a single integer value, not a list
# num_epochs = 300
# lr = .0002

# # Initialize the models, optimizers, and loss lists
# dLoss = []
# gLoss = []
# G = GeneratorGAN(kernSize=5, padVal=2)
# G = G.to(device)
# G.apply(w_init)
# D = DiscriminatorGAN(kernSize=5, padVal=2)
# D = D.to(device)
# D.apply(w_init)
# optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
# optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))

# # Train the models
# trainProcess(num_epochs, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 5, 2, device)
# trainResults(num_epochs, G, dLoss, gLoss, device)



![image-2.png](attachment:image-2.png) <br>
![image-3.png](attachment:image-3.png) <br>
![image.png](attachment:image.png) <br>
![image-4.png](attachment:image-4.png) <br>
![image-5.png](attachment:image-5.png) <br>

### Optimizing Learning Rate

In [ ]:
epoch_num = 100
# learning_rates = [.00009, .0006, .0009]
# the parameters below reset every time to ensure a new model is being trained after every iteration

# for lr in learning_rates:
#     dLoss = []
#     gLoss = []
#     G = GeneratorGAN(kernSize=5, padVal=2)
#     G = G.to(device)
#     G.apply(w_init)
#     D = DiscriminatorGAN(kernSize=5, padVal=2)
#     D = D.to(device)
#     D.apply(w_init)
#     optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
#     optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))
#     trainProcess(epoch_num, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 5, 2, device)
#     trainResults(epoch_num, G, dLoss, gLoss, device)

# lr = .000009

# dLoss = []
# gLoss = []
# G = GeneratorGAN(kernSize=5, padVal=2)
# G = G.to(device)
# G.apply(w_init)
# D = DiscriminatorGAN(kernSize=5, padVal=2)
# D = D.to(device)
# D.apply(w_init)
# optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
# optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))

# # Train the models
# trainProcess(num_epochs, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 5, 2, device)
# trainResults(num_epochs, G, dLoss, gLoss, device)

In [ ]:
# epoch_num = 300
# lr = .00009

# dLoss = []
# gLoss = []
# G = GeneratorGAN(kernSize=5, padVal=2)
# G = G.to(device)
# G.apply(w_init)
# D = DiscriminatorGAN(kernSize=5, padVal=2)
# D = D.to(device)
# D.apply(w_init)
# optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
# optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))

# # Train the models
# trainProcess(num_epochs, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 5, 2, device)
# trainResults(num_epochs, G, dLoss, gLoss, device)

![image.png](attachment:image.png) <br>
![image-2.png](attachment:image-2.png)

### Trying a Smaller Kernel Size

In [ ]:
# epoch_num = 100
# lr = .00009
# batch_size = 128
# dLoss = []
# gLoss = []
# G = GeneratorGAN(kernSize=3, padVal=1)
# G = G.to(device)
# G.apply(w_init)
# D = DiscriminatorGAN(kernSize=3, padVal=1)
# D = D.to(device)
# D.apply(w_init)
# optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
# optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))

# trainProcess(epoch_num, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 3, 1, device)
# trainResults(epoch_num, G, dLoss, gLoss, device)


### Trying a Larger Kernel Size

In [ ]:
# epoch_num = 300
# lr = .00009
# batch_size = 128
# dLoss = []
# gLoss = []
# G = GeneratorGAN(kernSize=7, padVal=3)
# G = G.to(device)
# G.apply(w_init)
# D = DiscriminatorGAN(kernSize=7, padVal=3)
# D = D.to(device)
# D.apply(w_init)
# optimD = optim.Adam(D.parameters(), lr=lr, betas=(0.5, .999))
# optimG = optim.Adam(G.parameters(), lr=lr, betas=(0.5, .999))
# trainProcess(epoch_num, train_loader, D, G, optimD, optimG, dLoss, gLoss, lr, 7, 3, device)
# trainResults(epoch_num, G, dLoss, gLoss, device)



### Testing

#### 200 Epochs, KernelSize 5, Padding 2

In [ ]:
fakeResults, realResults = testResults(test_loader, 200, .00009, 5, 2)
testPlotting(fakeResults, realResults)

#### 300 Epochs, KernelSize 5, Padding 2

In [ ]:
fakeResults, realResults = testResults(test_loader, 300, .00009, 5, 2)
testPlotting(fakeResults, realResults)

![img](images/300epochsconfmtx.png)